In [ ]:
import subprocess

In [ ]:
configs = ['4n','1f3n','2f2n','3f1n','4f']
iterations = 100

In [ ]:
for config in configs:
    print(config)
    for i in range(0, iterations):
        subprocess.run(['./market-des', '-r figgie -s 10000', '--index '+str(i),
                       '--output '+config, '--config '+config],stdout=subprocess.DEVNULL)

In [ ]:
import matplotlib.pyplot as plt
import pickle
import numpy as np
import os
import math

class Trader:
    pass

class CustomUnpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if name == 'Trader':
            return Trader
        return super().find_class(module, name)

In [ ]:
count = 0

for config in configs:
    print(config)
    endcash = np.zeros([iterations,4])
    for i in range(0, iterations):
        #print(i)
        filename = os.path.join('./output',config+'.'+str(i))
        data = CustomUnpickler(open(filename, 'rb')).load()
        cash = [agent.cash for agent in data['agents']]
        names = [agent.name for agent in data['agents']]
        endcash[i,:] = cash
        #print(len(data['assets'][0].sells))
        #print(data['agents'][0].cash)
    means = np.mean(endcash,axis=0)
    stds  = np.std(endcash,axis=0)
    
    fig, ax = plt.subplots()
    color_f = ['skyblue'] * count
    color_n = ['palegreen'] * (4-count)
    count+=1
    color = color_f + color_n
    ax.bar(range(0,4),means,color = color)
    ax.set_ylim((0,600))
    ax.set_ylabel("Mean cash + payout at end")
    ax.set_xlabel("Player")
    ax.errorbar(range(0,4), means, yerr=stds*2/math.sqrt(iterations), fmt="none", color="black")
    ax.set_xticks(range(0,4))
    ax.set_xticklabels(names)
    
    plt.show()
    fig.savefig('output/'+config+".png", facecolor='white', transparent=False)